In [ ]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import os 
from tqdm import tqdm
import glob
import tensorflow as tf
import keras
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

import cv2
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import gray2rgb


from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.utils import to_categorical
from keras import optimizers
from tensorflow.keras import optimizers 
from keras.callbacks import Callback,ModelCheckpoint,ReduceLROnPlateau
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

from typeguard import typechecked
from typing import Optional

from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model, load_model
from keras.layers import ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import matplotlib.pyplot as plt

import itertools

AUTOTUNE = tf.data.experimental.AUTOTUNE

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True, vertical_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255)


In [ ]:
train_dataset  = train_datagen.flow_from_directory(directory = '../input/bigdataset/CLASSIFY_DATA/train',
                                                   target_size = (224,224),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   batch_size = 16,
                                                   shuffle = True)


test_dataset = test_datagen.flow_from_directory(directory = '../input/bigdataset/CLASSIFY_DATA/test',
                                                target_size = (224,224),
                                                class_mode = 'categorical',
                                                batch_size = 16, shuffle=False)

In [ ]:
Label0={'NORMAL':0,'PNEUMONIA':1,'COVID':2}
Label1={'PNEUMONIA':0,'COVID':1,'NORMAL':2}
Label2={'COVID':0,'NORMAL':1,'PNEUMONIA':2}

# convert label to code
def getCode0(label):
    return Label0[label]

def getCode1(label):
    return Label1[label]

def getCode2(label):
    return Label2[label]


# convert code to label 
def getLabel0(n):
    for x,c in Label0.items():
        if n==c:
            return x

def getLabel1(n):
    for x,c in Label1.items():
        if n==c:
            return x
        
def getLabel2(n):
    for x,c in Label2.items():
        if n==c:
            return x
        
        
        
#Test        
print(getCode1('COVID'))
print(getLabel2(1))

In [ ]:
# Import the Xception model

from tensorflow.keras.applications.xception import Xception

In [ ]:
# Define your pre-trained model

input_shape = (224,224,3)
pretrained_model = Xception(input_shape = input_shape,
                weights = 'imagenet',pooling='max',
                include_top = False)

In [ ]:
for layer in pretrained_model.layers[:45]:
    layer.trainable = False

for i, layer in enumerate(pretrained_model.layers):
# Make sure you have frozen the correct layers
    print(i, layer.name, layer.trainable)

In [ ]:
# Define your model

model = Sequential()

model.add(pretrained_model)

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(3))
model.add(Activation('softmax'))


#model.summary() 

In [ ]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),
           tf.keras.metrics.Precision(name='precision'),
           tf.keras.metrics.Recall(name='recall'),  
           tf.keras.metrics.AUC(name='auc'),
           f1_score,
          ]

In [ ]:
# Define early stopping and model checkpoint for optimizing epoch number and saving the best model

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
mc = ModelCheckpoint('covid_19_xception2.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
lrd = ReduceLROnPlateau(monitor = 'val_loss',patience = 25,verbose = 1,factor = 0.7)

In [ ]:
# Compile and fit your model

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=METRICS)

epochs = 50
history = model.fit_generator(train_dataset,
                              steps_per_epoch=len(train_dataset)//16,
                              epochs=epochs,
                              validation_data=test_dataset,
                              validation_steps=len(test_dataset)//16,
                              callbacks=[es,mc,lrd] )

In [ ]:
# Plot accuracy and loss for testing and validation

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy Value')
plt.xlabel('Epoch')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Loss Value')
plt.xlabel('Epoch')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
scores = model.evaluate_generator(test_dataset)

In [ ]:
print("Accuracy = ", scores[1])
print("Precision = ", scores[2])
print("Recall = ", scores[3])
print("AUC = ", scores[4])
print("F1_score = ", scores[5])

In [ ]:
batchSize=16
test_dataset.reset()
testStep = (test_dataset.samples + (batchSize-1)) // batchSize
print("testStep: ", testStep)
predictions = model.predict_generator(test_dataset, steps = testStep ,  verbose = 1)
len(predictions)

In [ ]:
predicted_class_indices=np.argmax(predictions,axis=1) #along the specified axis
print(predicted_class_indices)
len(predicted_class_indices)

In [ ]:
labels = (test_dataset.class_indices)
print(labels)

labels = dict((v,k) for k,v in labels.items())
print(labels)

In [ ]:
predictedLables= [labels[k] for k in predicted_class_indices]
len(predictedLables)

In [ ]:
actualLables= [labels[k] for k in test_dataset.classes]
len(actualLables)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(actualLables, predictedLables)

In [ ]:
matrix = confusion_matrix(actualLables, predictedLables)
print(labels)
matrix

In [ ]:
print(classification_report(actualLables, predictedLables))

In [ ]:
#Prepared code that is taken from SKLearn Website, Creates Confusion Matrix
def plot_confusion_matrix(cm, classes,
                          normalize=True,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="black" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.xlabel('True label')
    plt.ylabel('Predicted label')

In [ ]:
cm_plot_labels = ['COVID', 'NON-COVID', 'PNEUMONIA']
plot_confusion_matrix(matrix,cm_plot_labels, normalize=False
                      , title = 'Confusion Matrix')

In [ ]:
#prediction
pred=model.predict(test_dataset)

In [ ]:
print(len(pred))

In [ ]:
y_test=[]
for i in range(10):
    y_test.extend(test_dataset.__getitem__(i)[1])


In [ ]:
y_test=np.argmax(y_test,axis=1)
pred= np.argmax(pred,axis=1)

In [ ]:
print("pred \n",len(pred))
print("y_test \n",len(y_test))

In [ ]:
print("y_test \n",y_test)
print("pred \n",pred)

In [ ]:
#covid
import cv2
plt.figure(figsize=(20,10))
for i in range(0,9):
    
    plt.subplot(3,3,i+1 )
    
    plt.imshow(test_dataset.__getitem__(0)[0][i],cmap='gray')
    plt.title(f"   Real: {getLabel2(y_test[i])   } Vs  Predict: {getLabel2(pred[i])}")